# Notebook: Compare LLM and Human Annotations

## Packages

In [256]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import numpy as np
import json
import sys
import os

In [257]:
sys.path.append(os.path.abspath('../07 train models/'))
from TASD.evaluation import calculate_metrics_for_examples
import constants

## Settings

In [258]:
LLMS = ["Llama70B", "GPT-3"]
FEW_SHOT_CONDITIONS = ["fixed", "random"]

In [259]:
LLMS_ENCODED = {"GPT-3": "\\textbf{GPT-3.5-turbo}", "Llama70B": "\\textbf{Llama-2-70B}"}
ENCODE_CONDITION = {"fixed": "\\textbf{LRS\\textsubscript{25}}",
                    "random": "\\textbf{LRS\\textsubscript{500}}"}

## Code

### Load Data

#### Human Annotations

In [260]:
with open(f"annotation_datasets/annotated_synth_dataset.json", 'r') as json_file:
    human_annotations = json.load(json_file)    

#### Load Synthetic 

In [261]:
llm_annotations = []

for llm in LLMS:
    for fs in ["random", "fixed"]:
       for split_id in range(6):
           with open(f"../07 train models/synth/{llm}/{fs}/split_{split_id}.json", 'r') as json_file:
              synthetic_data_split = json.load(json_file)
              for example in  synthetic_data_split:
                  llm_annotations.append(example)    

In [262]:
llm_annotations_aspects = [([{"aspect_category": tag["label"], "aspect_polarity": tag["polarity"],
                              "aspect_term": tag["text"] if tag["text"] != 'NULL' else None, "start": tag["start"], "end": tag["end"]} for tag in example["tags"]], example["id"]) for example in llm_annotations]
human_annotations_aspects = [([{"aspect_category": tag["label"], "aspect_polarity": tag["polarity"], "aspect_term": tag["text"]
                                if tag["text"] != 'NULL' else None, "start": tag["start"], "end": tag["end"]} for tag in example["tags"]], example["id"], example["model"], example["few_shot_condtion"]) for example in human_annotations]

In [263]:
dataset = {}

In [264]:
def get_example_with_id(id, dataset):
    return [example for example in dataset if example[1] == id][0][0]

for llm in LLMS:
    dataset[llm] = {}
    for fs in FEW_SHOT_CONDITIONS:
        dataset[llm][fs] = {}
        human_annotations_aspects_ids = [example[1] for example in human_annotations_aspects if example[2] == llm and example[3] == fs]
        human_annotations_samples = [example[0] for example in human_annotations_aspects if example[2] == llm and example[3] == fs]
        llm_annotations_samples = [get_example_with_id(id, llm_annotations_aspects) for id in human_annotations_aspects_ids]

        dataset[llm][fs]["human_annotation"] = human_annotations_samples
        dataset[llm][fs]["llm_annotation"] = llm_annotations_samples

### Analyse Quality

#### Aspect Term Detection

In [265]:
def calculate_tp_tn_fp_fn_aspect_term(pred, label):
    pred_set = set(
        f"{range['start']}_{range['end']}" for range in pred)
    label_set = set(
        f"{range['start']}_{range['end']}" for range in label)

    tp_set = pred_set & label_set
    tp = len(tp_set)

    fp_set = pred_set - tp_set
    fp = len(fp_set)

    fn_set = label_set - tp_set
    fn = len(fn_set)

    return tp, 0, fp, fn


In [266]:
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        llm_annotations_aspect_terms = [
            [{"start": tag["start"], "end": tag["end"]} for tag in example if tag["aspect_term"] is not None] for example in dataset[llm][fs]["llm_annotation"]]
        human_annotations_aspect_terms = [
            [{"start": tag["start"], "end": tag["end"]} for tag in example if tag["aspect_term"] is not None] for example in dataset[llm][fs]["human_annotation"]]

        tp_total = tn_total = fp_total = fn_total = 0
        for i in range(len(human_annotations_aspect_terms)):
            tp, tn, fp, fn = calculate_tp_tn_fp_fn_aspect_term(
                llm_annotations_aspect_terms[i], human_annotations_aspect_terms[i])
            tp_total += tp
            tn_total += tn
            fp_total += fp
            fn_total += fn

        # Calculate metrics
        accuracy = (tp_total + tn_total) / (tp_total + tn_total + fp_total +
                                            fn_total) if (tp_total + tn_total + fp_total + fn_total) > 0 else 0
        precision = tp_total / \
            (tp_total + fp_total) if (tp_total + fp_total) > 0 else 0
        recall = tp_total / \
            (tp_total + fn_total) if (tp_total + fn_total) > 0 else 0

        f1 = 2 * tp_total / (2 * tp_total + fn_total + fp_total)

        llm_print = "\\multirow{2}{*}{" + \
            LLMS_ENCODED[llm] + "}" if fs_idx == 0 else ""

        fs_print = ENCODE_CONDITION[fs]

        print(llm_print, "&", fs_print,
              "&", "{:.2f}".format(f1*100),
              "&", "{:.2f}".format(accuracy*100),
              "&", "{:.2f}".format(precision*100),
              "&", "{:.2f}".format(recall*100), "\\\\")
    print("\hline")

\multirow{2}{*}{\textbf{Llama-2-70B}} & \textbf{LRS\textsubscript{25}} & 80.59 & 67.48 & 83.71 & 77.68 \\
 & \textbf{LRS\textsubscript{500}} & 79.52 & 66.00 & 81.48 & 77.66 \\
\hline
\multirow{2}{*}{\textbf{GPT-3.5-turbo}} & \textbf{LRS\textsubscript{25}} & 75.94 & 61.22 & 91.41 & 64.95 \\
 & \textbf{LRS\textsubscript{500}} & 73.72 & 58.38 & 93.60 & 60.81 \\
\hline


#### Aspect Category

In [267]:
def category_list_to_label(cat_list):
    return [1 if cat in cat_list else 0 for cat in constants.ASPECT_CATEGORIES]

In [268]:
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        llm_annotations_aspect_categories = [category_list_to_label(
            [tag["aspect_category"] for tag in example]) for example in dataset[llm][fs]["llm_annotation"]]
        human_annotations_aspect_categories = [category_list_to_label(
            [tag["aspect_category"] for tag in example]) for example in dataset[llm][fs]["human_annotation"]]

        accuracy = accuracy_score(
            human_annotations_aspect_categories, llm_annotations_aspect_categories)
        f1_micro = f1_score(human_annotations_aspect_categories,
                            llm_annotations_aspect_categories, average='micro')
        f1_macro = f1_score(human_annotations_aspect_categories,
                            llm_annotations_aspect_categories, average='macro')

        llm_print = "\\multirow{2}{*}{" + \
            LLMS_ENCODED[llm] + "}" if fs_idx == 0 else ""

        print(llm_print, "&", ENCODE_CONDITION[fs],
              "&", "{:.2f}".format(f1_micro*100),
              "&", "{:.2f}".format(f1_macro*100),
              "&", "{:.2f}".format(accuracy*100), "\\\\")
    print("\\hline")

\multirow{2}{*}{\textbf{Llama-2-70B}} & \textbf{LRS\textsubscript{25}} & 74.80 & 74.00 & 58.67 \\
 & \textbf{LRS\textsubscript{500}} & 77.93 & 77.02 & 65.33 \\
\hline
\multirow{2}{*}{\textbf{GPT-3.5-turbo}} & \textbf{LRS\textsubscript{25}} & 95.82 & 95.63 & 91.67 \\
 & \textbf{LRS\textsubscript{500}} & 95.06 & 95.13 & 90.67 \\
\hline


#### Aspect Category (performance for each Aspect Category)

In [269]:
def category_list_to_label(cat_list):
    return [1 if cat in cat_list else 0 for cat in constants.ASPECT_CATEGORIES]


for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        llm_annotations_aspect_categories = [category_list_to_label(
            [tag["aspect_category"] for tag in example]) for example in dataset[llm][fs]["llm_annotation"]]
        human_annotations_aspect_categories = [category_list_to_label(
            [tag["aspect_category"] for tag in example]) for example in dataset[llm][fs]["human_annotation"]]

        for ac_idx, aspect_category in enumerate(constants.ASPECT_CATEGORIES):
            idx = constants.ASPECT_CATEGORIES.index(aspect_category)

            tp = sum((llm_annotations_aspect_categories[i][idx] == 1) and (
                human_annotations_aspect_categories[i][idx] == 1) for i in range(len(llm_annotations_aspect_categories)))
            fp = sum((llm_annotations_aspect_categories[i][idx] == 1) and (
                human_annotations_aspect_categories[i][idx] == 0) for i in range(len(llm_annotations_aspect_categories)))
            fn = sum((llm_annotations_aspect_categories[i][idx] == 0) and (
                human_annotations_aspect_categories[i][idx] == 1) for i in range(len(llm_annotations_aspect_categories)))

            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            accuracy = accuracy_score([example[idx] for example in human_annotations_aspect_categories], [
                                      example[idx] for example in llm_annotations_aspect_categories])

            f1 = 2 * tp / (2 * tp + fn + fp)

            n_samples_in_class = sum(
                example[idx] == 1 for example in human_annotations_aspect_categories)

            llm_print = "\\multirow{10}{*}{" + \
                LLMS_ENCODED[llm] + "}" if fs_idx == 0 and ac_idx == 0 else ""
            fs_print = "\\multirow{5}{*}{" + \
                ENCODE_CONDITION[fs] + "}" if ac_idx == 0 else ""

            print(llm_print, "&", fs_print,
                  "&", "\\texttt{"+aspect_category+"}",
                  "&", "{:.2f}".format(f1*100),
                  "&", "{:.2f}".format(accuracy*100),
                  "&", "{:.2f}".format(precision*100),
                  "&", "{:.2f}".format(recall*100), "\\\\")

        print("\\arrayrulecolor{gray}\cline{2-7}\\arrayrulecolor{black}")

    print("\\hline")

\multirow{10}{*}{\textbf{Llama-2-70B}} & \multirow{5}{*}{\textbf{LRS\textsubscript{25}}} & \texttt{GENERAL-IMPRESSION} & 54.68 & 79.83 & 42.94 & 75.26 \\
 &  & \texttt{FOOD} & 74.66 & 84.50 & 82.04 & 68.50 \\
 &  & \texttt{SERVICE} & 80.23 & 88.50 & 75.27 & 85.89 \\
 &  & \texttt{AMBIENCE} & 77.67 & 88.50 & 77.92 & 77.42 \\
 &  & \texttt{PRICE} & 82.78 & 90.50 & 75.27 & 91.95 \\
\arrayrulecolor{gray}\cline{2-7}\arrayrulecolor{black}
 & \multirow{5}{*}{\textbf{LRS\textsubscript{500}}} & \texttt{GENERAL-IMPRESSION} & 56.67 & 82.67 & 42.50 & 85.00 \\
 &  & \texttt{FOOD} & 72.73 & 85.00 & 80.54 & 66.30 \\
 &  & \texttt{SERVICE} & 84.50 & 91.50 & 85.28 & 83.73 \\
 &  & \texttt{AMBIENCE} & 83.77 & 91.67 & 80.62 & 87.16 \\
 &  & \texttt{PRICE} & 87.42 & 93.67 & 80.49 & 95.65 \\
\arrayrulecolor{gray}\cline{2-7}\arrayrulecolor{black}
\hline
\multirow{10}{*}{\textbf{GPT-3.5-turbo}} & \multirow{5}{*}{\textbf{LRS\textsubscript{25}}} & \texttt{GENERAL-IMPRESSION} & 92.70 & 96.17 & 88.48 & 97.33 \\


#### Aspect Category + Sentiment Polarity

In [270]:
AC_POLARITY_COMBINATIONS = [cat+"_"+polarity for cat in constants.ASPECT_CATEGORIES for polarity in ["POSITIVE", "NEGATIVE", "NEUTRAL", "CONFLICT"]]

In [271]:
def category_polarity_list_to_label(cat_pol_list):
    return [1 if ac_pol in cat_pol_list else 0 for ac_pol in AC_POLARITY_COMBINATIONS]

In [272]:
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        llm_annotations_ac_pol = [category_polarity_list_to_label(
            [tag["aspect_category"]+"_"+tag["aspect_polarity"] for tag in example]) for example in dataset[llm][fs]["llm_annotation"]]
        human_annotations_ac_pol = [category_polarity_list_to_label(
            [tag["aspect_category"]+"_"+tag["aspect_polarity"] for tag in example]) for example in dataset[llm][fs]["human_annotation"]]

        accuracy = accuracy_score(
            human_annotations_ac_pol, llm_annotations_ac_pol)
        f1_micro = f1_score(human_annotations_ac_pol,
                            llm_annotations_ac_pol, average='micro', zero_division=0)
        f1_macro = f1_score(human_annotations_ac_pol,
                            llm_annotations_ac_pol, average='macro', zero_division=0)

        llm_print = "\\multirow{2}{*}{" + \
            LLMS_ENCODED[llm] + "}" if fs_idx == 0 else ""

        print(llm_print,
              "&", ENCODE_CONDITION[fs],
              "&", "{:.2f}".format(f1_micro*100),
              "&", "{:.2f}".format(f1_macro*100),
              "&", "{:.2f}".format(accuracy*100), "\\\\")
    print("\\hline")

\multirow{2}{*}{\textbf{Llama-2-70B}} & \textbf{LRS\textsubscript{25}} & 50.09 & 35.03 & 37.33 \\
 & \textbf{LRS\textsubscript{500}} & 53.32 & 37.98 & 43.50 \\
\hline
\multirow{2}{*}{\textbf{GPT-3.5-turbo}} & \textbf{LRS\textsubscript{25}} & 84.08 & 62.93 & 75.00 \\
 & \textbf{LRS\textsubscript{500}} & 83.84 & 62.10 & 79.17 \\
\hline


In [273]:
def category_polarity_list_to_label(cat_polarity_list):
    return [1 if cat_polarity in cat_polarity_list else 0 for cat_polarity in AC_POLARITY_COMBINATIONS]


for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        llm_annotations_ac_pol = [category_polarity_list_to_label(
            [tag["aspect_category"] + "_" + tag["aspect_polarity"] for tag in example]) for example in dataset[llm][fs]["llm_annotation"]]
        human_annotations_ac_pol = [category_polarity_list_to_label(
            [tag["aspect_category"] + "_" + tag["aspect_polarity"] for tag in example]) for example in dataset[llm][fs]["human_annotation"]]

        for ac_pol_idx, ac_pol_combination in enumerate(AC_POLARITY_COMBINATIONS):
            idx = AC_POLARITY_COMBINATIONS.index(ac_pol_combination)

            tp = sum((llm_annotations_ac_pol[i][idx] == 1) and (
                human_annotations_ac_pol[i][idx] == 1) for i in range(len(llm_annotations_ac_pol)))
            fp = sum((llm_annotations_ac_pol[i][idx] == 1) and (
                human_annotations_ac_pol[i][idx] == 0) for i in range(len(llm_annotations_ac_pol)))
            fn = sum((llm_annotations_ac_pol[i][idx] == 0) and (
                human_annotations_ac_pol[i][idx] == 1) for i in range(len(llm_annotations_ac_pol)))

            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            accuracy = accuracy_score([example[idx] for example in human_annotations_ac_pol], [
                                      example[idx] for example in llm_annotations_ac_pol])
            try:
                f1 = 2 * tp / (2 * tp + fn + fp)
            except:
                f1 = 0

            n_samples_in_class = sum(
                example[idx] == 1 for example in human_annotations_ac_pol)

            aspect_category, sentiment_polarity = ac_pol_combination.split("_")

            llm_print = "\\multirow{30}{*}{" + \
                LLMS_ENCODED[llm] + \
                "}" if fs_idx == 0 and ac_pol_idx == 0 else ""
            fs_print = "\\multirow{15}{*}{" + \
                ENCODE_CONDITION[fs] + "}" if ac_pol_idx == 0 else ""
            ac_print = "\\multirow{3}{*}{" + "\\texttt{" + \
                aspect_category+"}" + "}" if ac_pol_idx % 3 == 0 else ""

            if sentiment_polarity != "CONFLICT":
                print(llm_print,
                      "&", fs_print,
                      "&", ac_print,
                      "&", "\\texttt{"+sentiment_polarity+"}",
                      "&", "{:.2f}".format(f1*100),
                      "&", "{:.2f}".format(accuracy*100),
                      "&", "{:.2f}".format(precision*100),
                      "&", "{:.2f}".format(recall*100), "\\\\")

            if ac_pol_idx % 4 == 2:
                print(
                    "\\arrayrulecolor{gray}\cline{3-8}\\arrayrulecolor{black}")

        print("\\cline{2-8}")
    print("\\hline")

\multirow{30}{*}{\textbf{Llama-2-70B}} & \multirow{15}{*}{\textbf{LRS\textsubscript{25}}} & \multirow{3}{*}{\texttt{GENERAL-IMPRESSION}} & \texttt{POSITIVE} & 49.18 & 89.67 & 50.85 & 47.62 \\
 &  &  & \texttt{NEGATIVE} & 29.63 & 90.50 & 21.05 & 50.00 \\
 &  &  & \texttt{NEUTRAL} & 17.50 & 89.00 & 9.86 & 77.78 \\
\arrayrulecolor{gray}\cline{3-8}\arrayrulecolor{black}
 &  &  & \texttt{POSITIVE} & 55.76 & 87.83 & 77.97 & 43.40 \\
 &  &  & \texttt{NEGATIVE} & 62.07 & 92.67 & 69.23 & 56.25 \\
 &  & \multirow{3}{*}{\texttt{FOOD}} & \texttt{NEUTRAL} & 42.86 & 90.67 & 32.31 & 63.64 \\
\arrayrulecolor{gray}\cline{3-8}\arrayrulecolor{black}
 &  &  & \texttt{POSITIVE} & 58.28 & 89.50 & 72.13 & 48.89 \\
 &  & \multirow{3}{*}{\texttt{SERVICE}} & \texttt{NEGATIVE} & 67.74 & 93.33 & 66.67 & 68.85 \\
 &  &  & \texttt{NEUTRAL} & 21.95 & 89.33 & 12.33 & 100.00 \\
\arrayrulecolor{gray}\cline{3-8}\arrayrulecolor{black}
 &  & \multirow{3}{*}{\texttt{AMBIENCE}} & \texttt{POSITIVE} & 49.23 & 89.00 & 69.57 & 

#### Aspect Term + Sentiment Polarity

In [274]:
def calculate_tp_tn_fp_fn_e2e(pred, label):
    pred_set = set(
        f"{range['start']}_{range['end']}_{range['polarity']}" for range in pred)
    label_set = set(
        f"{range['start']}_{range['end']}_{range['polarity']}" for range in label)

    tp_set = pred_set & label_set
    tp = len(tp_set)

    fp_set = pred_set - tp_set
    fp = len(fp_set)

    fn_set = label_set - tp_set
    fn = len(fn_set)

    return tp, 0, fp, fn

In [275]:
def calculate_tp_tn_fp_fn_e2e_total(pred_total, label_total):
    tp_total = tn_total = fp_total = fn_total = 0
    for i in range(len(label_total)):
        tp, tn, fp, fn = calculate_tp_tn_fp_fn_e2e(
            pred_total[i], label_total[i])
        tp_total += tp
        tn_total += tn
        fp_total += fp
        fn_total += fn
    return tp_total, tn_total, fp_total, fn_total

In [276]:
def f1_macro_e2e(llm_annotations_aspect_terms, human_annotations_aspect_terms):
    f1_scores = []
    for pol in ["POSITIVE", "NEGATIVE", "NEUTRAL", "CONFLICT"]:
        llm_annotations_class = [[tag for tag in example if tag["polarity"] == pol]
                                 for example in llm_annotations_aspect_terms]
        human_annotations_class = [[tag for tag in example if tag["polarity"] == pol]
                                   for example in human_annotations_aspect_terms]
        
        tp_total, tn_total, fp_total, fn_total = calculate_tp_tn_fp_fn_e2e_total(llm_annotations_class, human_annotations_class)
        f1_scores.append(2 * tp_total / (2 * tp_total + fn_total + fp_total))
    return np.mean(f1_scores)

In [277]:
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        llm_annotations_aspect_terms = [
            [{"start": tag["start"], "end": tag["end"], "polarity": tag["aspect_polarity"]} for tag in example if tag["aspect_term"] is not None] for example in dataset[llm][fs]["llm_annotation"]]
        human_annotations_aspect_terms = [
            [{"start": tag["start"], "end": tag["end"], "polarity": tag["aspect_polarity"]} for tag in example if tag["aspect_term"] is not None] for example in dataset[llm][fs]["human_annotation"]]

        tp_total, tn_total, fp_total, fn_total = calculate_tp_tn_fp_fn_e2e_total(
            llm_annotations_aspect_terms, human_annotations_aspect_terms)

        # Calculate metrics
        accuracy = (tp_total + tn_total) / (tp_total + tn_total + fp_total +
                                            fn_total) if (tp_total + tn_total + fp_total + fn_total) > 0 else 0
        precision = tp_total / \
            (tp_total + fp_total) if (tp_total + fp_total) > 0 else 0
        recall = tp_total / \
            (tp_total + fn_total) if (tp_total + fn_total) > 0 else 0

        f1_micro = 2 * tp_total / (2 * tp_total + fn_total + fp_total)

        f1_macro = f1_macro_e2e(
            llm_annotations_aspect_terms, human_annotations_aspect_terms)

        llm_print = "\\multirow{2}{*}{" + \
            LLMS_ENCODED[llm] + "}" if fs_idx == 0 else ""

        print(llm_print, "&", ENCODE_CONDITION[fs],
              "&", "{:.2f}".format(f1_micro*100),
              "&", "{:.2f}".format(f1_macro*100),
              "&", "{:.2f}".format(accuracy*100), "\\\\")
    print("\hline")

\multirow{2}{*}{\textbf{Llama-2-70B}} & \textbf{LRS\textsubscript{25}} & 54.56 & 38.66 & 37.51 \\
 & \textbf{LRS\textsubscript{500}} & 54.14 & 38.44 & 37.12 \\
\hline
\multirow{2}{*}{\textbf{GPT-3.5-turbo}} & \textbf{LRS\textsubscript{25}} & 67.64 & 50.49 & 51.11 \\
 & \textbf{LRS\textsubscript{500}} & 66.88 & 49.79 & 50.24 \\
\hline


#### Aspect Term + Aspect Category + Sentiment Polarity

In [278]:
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        metrics_triplets = calculate_metrics_for_examples(
            dataset[llm][fs]["human_annotation"], dataset[llm][fs]["llm_annotation"])

        llm_print = "\\multirow{2}{*}{" + \
            LLMS_ENCODED[llm] + "}" if fs_idx == 0 else ""
        print(llm_print,
              "&", ENCODE_CONDITION[fs],
              "&", "{:.2f}".format(metrics_triplets["f1"] * 100),
              "&", "{:.2f}".format(metrics_triplets["accuracy"] * 100),
              "&", "{:.2f}".format(metrics_triplets["precision"] * 100),
              "&", "{:.2f}".format(metrics_triplets["recall"] * 100), "\\\\")

    print("\\hline")

\multirow{2}{*}{\textbf{Llama-2-70B}} & \textbf{LRS\textsubscript{25}} & 40.25 & 25.20 & 36.74 & 44.50 \\
 & \textbf{LRS\textsubscript{500}} & 43.62 & 27.89 & 40.05 & 47.89 \\
\hline
\multirow{2}{*}{\textbf{GPT-3.5-turbo}} & \textbf{LRS\textsubscript{25}} & 56.68 & 39.55 & 55.39 & 58.03 \\
 & \textbf{LRS\textsubscript{500}} & 56.30 & 39.17 & 55.89 & 56.71 \\
\hline
